In [83]:
import os
SNAPPY_notebook_path = os.path.join(os.path.abspath(""), "bench.ipynb")

time: 21 ms (started: 2024-05-07 14:49:03 -04:00)


In [84]:
import time
SNAPPY_start_time = time.perf_counter_ns()

time: 766 µs (started: 2024-05-07 14:49:03 -04:00)


In [85]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 9.33 ms (started: 2024-05-07 14:49:03 -04:00)


# Exploratory Data Analysis

In this notebook we will look into the speedtest data gathered from ookla and draw some useful insights from the dataset. Additional dataset would also be utilized to make some meaningful insights into the appropriately combined wholistic data

## File Exploration

In the following block we see all the files available for this kernel. The relevant libraries are also imported to process the data extracted from the dataset.

In [86]:
# !pip install dias

time: 889 µs (started: 2024-05-07 14:49:03 -04:00)


In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import os
# STEFANOS: Conditionally import Modin Pandas
if "IREWR_WITH_MODIN" in os.environ and os.environ["IREWR_WITH_MODIN"] == "True":
    # STEFANOS: Import Modin Pandas
    import os
    os.environ["MODIN_ENGINE"] = "ray"
    import ray
    ray.init(num_cpus=int(os.environ['MODIN_CPUS']), runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})
    import modin.pandas as pd
else:
    # STEFANOS: Import regular Pandas
    import pandas as pd
import regex as re # For String searches
# import plotly.graph_objects as go
# import plotly.express as px
# Input data files are available in the read-only "input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(os.path.abspath('') +'/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/sam/dias-benchmarks/dias-benchmarks/notebooks/gksriharsha/eda-speedtests/input/speedtest-data-by-ookla/mobile_year_2020_quarter_02.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/gksriharsha/eda-speedtests/input/speedtest-data-by-ookla/mobile_year_2022_quarter_03.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/gksriharsha/eda-speedtests/input/speedtest-data-by-ookla/mobile_year_2022_quarter_01.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/gksriharsha/eda-speedtests/input/speedtest-data-by-ookla/mobile_year_2020_quarter_01.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/gksriharsha/eda-speedtests/input/speedtest-data-by-ookla/fixed_year_2021_quarter_01.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/gksriharsha/eda-speedtests/input/speedtest-data-by-ookla/fixed_year_2022_quarter_01.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/gksriharsha/eda-speedtests/input/speedtest-data-by-ookla/fixed_year_2020_quarter_02.csv
/home/sam/dias-b

In [88]:
data = pd.read_csv('./input/speedtest-data-by-ookla/fixed_year_2021_quarter_03.csv')
data.head()

,Name,Number of Records,Devices,Tests,Avg. Avg U Kbps,Avg. Avg D Kbps,Avg Lat Ms,Avg. Pop2005,Rank Upload,Rank Download,Rank Latency
0,Afghanistan,897,"2,312","9,003","3,983","4,823",70,"25,067,407",217,231,33
1,Åland Islands,284,580,"1,341","58,693","82,767",11,0,27,47,215
2,Albania,"7,215","38,905","104,752","17,354","25,803",20,"3,153,731",100,127,159
3,Algeria,"16,056","70,929","413,666","1,508","9,057",43,"32,854,159",234,211,68
4,American Samoa,80,202,900,"14,345","33,078",18,"64,051",119,104,171


time: 146 ms (started: 2024-05-07 14:49:03 -04:00)


In [89]:
# data.info()

time: 297 µs (started: 2024-05-07 14:49:03 -04:00)


In [90]:
Mobile_df = pd.DataFrame([],columns=data.columns)
Broadband_df = pd.DataFrame([],columns=data.columns)

def col_name_corrections(df,correction_pair):
    if set(df.columns).intersection(set(correction_pair.keys())):
        df.rename(columns=correction_pair,inplace=True)
    return df

for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        meta_info = filename.split('/')[-1]
        data = pd.read_csv(dirname+'/'+filename,thousands=r',').convert_dtypes()
        data = col_name_corrections(data,{'Number of Record':'Number of Records'})
        data['Year'] = np.int64(re.search('year_(.*)_quarter',meta_info).group(1))
        data['Quarter'] = np.int64(re.search('quarter_(.*).csv',meta_info).group(1))
        if 'mobile' in meta_info:
            Mobile_df = pd.concat([Mobile_df,data])
        else:
            Broadband_df = pd.concat([Broadband_df,data]) 
print(Broadband_df.shape)
print(Mobile_df.shape)
Mobile_df = Mobile_df.astype({'Year':np.int64,'Quarter':np.int64})
Broadband_df = Broadband_df.astype({'Year':np.int64,'Quarter':np.int64})
Mobile_df.sort_values(by=['Year','Quarter'],ascending=[True,True],inplace=True)
Broadband_df.sort_values(by=['Year','Quarter'],ascending=[True,True],inplace=True)

(2597, 13)
(2487, 13)
time: 193 ms (started: 2024-05-07 14:49:03 -04:00)


/tmp/ipykernel_12645/2027441289.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Mobile_df = pd.concat([Mobile_df,data])
/tmp/ipykernel_12645/2027441289.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Broadband_df = pd.concat([Broadband_df,data])


It can be seen that there are more broadband rows than Mobile rows. This is a point that should be noted each row corresponds to a country's statistics in the particular year and quarter. Missing data indicates lack of speed test data from the country.

In [91]:
# Mobile_df.info()

time: 278 µs (started: 2024-05-07 14:49:03 -04:00)


# -- STEFANOS -- Replicate Data

In [92]:
factor = 2000
Mobile_df = pd.concat([Mobile_df]*factor, ignore_index=True)
# Mobile_df.info()

time: 971 ms (started: 2024-05-07 14:49:03 -04:00)


In [93]:
factor = 1000
Broadband_df = pd.concat([Broadband_df]*factor, ignore_index=True)
# Broadband_df.info()

time: 161 ms (started: 2024-05-07 14:49:04 -04:00)


In [94]:
Mobile_df.head()

,Name,Number of Records,Devices,Tests,Avg. Avg U Kbps,Avg. Avg D Kbps,Avg Lat Ms,Avg. Pop2005,Rank Upload,Rank Download,Rank Latency,Year,Quarter
0,Afghanistan,789,1241,3541,3099,6387,70,25067407,220,220,32,2020,1
1,Albania,1626,4513,7228,11414,40238,26,3153731,75,42,197,2020,1
2,Algeria,14174,48699,124160,6150,7724,65,32854159,197,213,35,2020,1
3,American Samoa,23,33,63,8488,22792,62,64051,156,111,38,2020,1
4,Andorra,45,83,110,18160,72764,39,73483,9,6,112,2020,1


time: 6.07 ms (started: 2024-05-07 14:49:04 -04:00)


In [95]:
unique_countries_broadband = Broadband_df.groupby('Name').count()
unique_countries_broadband.head()

,Number of Records,Devices,Tests,Avg. Avg U Kbps,Avg. Avg D Kbps,Avg Lat Ms,Avg. Pop2005,Rank Upload,Rank Download,Rank Latency,Year,Quarter
Name,,,,,,,,,,,,
Afghanistan,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000
Albania,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000
Algeria,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000
American Samoa,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000
Andorra,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000,11000


time: 167 ms (started: 2024-05-07 14:49:04 -04:00)


In [96]:
unique_countries_mobile = Mobile_df.groupby('Name').count()
unique_countries_mobile.head()

,Number of Records,Devices,Tests,Avg. Avg U Kbps,Avg. Avg D Kbps,Avg Lat Ms,Avg. Pop2005,Rank Upload,Rank Download,Rank Latency,Year,Quarter
Name,,,,,,,,,,,,
Afghanistan,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000
Albania,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000
Algeria,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000
American Samoa,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000
Andorra,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000


time: 312 ms (started: 2024-05-07 14:49:05 -04:00)


## Insights

The following countries do have mobile speedtest data for all the years and quarters, thereby making less than 10 reports (one per quarter).

In [97]:
# Check for missing values
Mobile_df.isna().any()

Name                 False
Number of Records    False
Devices              False
Tests                False
Avg. Avg U Kbps      False
Avg. Avg D Kbps      False
Avg Lat Ms           False
Avg. Pop2005         False
Rank Upload          False
Rank Download        False
Rank Latency         False
Year                 False
Quarter              False
dtype: bool

time: 106 ms (started: 2024-05-07 14:49:05 -04:00)


In [98]:
Broadband_df.isna().any()

Name                 False
Number of Records    False
Devices              False
Tests                False
Avg. Avg U Kbps      False
Avg. Avg D Kbps      False
Avg Lat Ms           False
Avg. Pop2005         False
Rank Upload          False
Rank Download        False
Rank Latency         False
Year                 False
Quarter              False
dtype: bool

time: 55.8 ms (started: 2024-05-07 14:49:05 -04:00)


In [99]:
unique_countries_mobile[unique_countries_mobile.Year < 10]['Year']

Series([], Name: Year, dtype: int64)

time: 2.44 ms (started: 2024-05-07 14:49:05 -04:00)


In [100]:
unique_countries_broadband[unique_countries_broadband.Year < 10]['Year']

Series([], Name: Year, dtype: int64)

time: 11.6 ms (started: 2024-05-07 14:49:05 -04:00)


## Raw Download Speed Visualization

This visualization can be used to show change of values per country. The improvement values cannot be understood by laymen because an improvement of 50 Kbps **national average** (given) means differnt things to different countries based on economy, population, GDP, Infrastructure, etc.

In [101]:
# DIAS_VERBOSE
Mobile_Stats = Mobile_df.groupby('Name').agg(
    Change_Download=('Avg. Avg D Kbps', lambda x: list(x)[-1] - list(x)[0]),
    Change_Upload=('Avg. Avg U Kbps', lambda x: list(x)[-1] - list(x)[0]),
    Change_Latency=('Avg Lat Ms', lambda x: list(x)[-1] - list(x)[0])
)
Broadband_Stats = Broadband_df.groupby('Name').agg(
    Change_Download=('Avg. Avg D Kbps', lambda x: list(x)[-1] - list(x)[0]),
    Change_Upload=('Avg. Avg U Kbps', lambda x: list(x)[-1] - list(x)[0]),
    Change_Latency=('Avg Lat Ms', lambda x: list(x)[-1] - list(x)[0])
)
# fig = px.histogram(Mobile_Stats['Change_Download'],title='Frequency distribution of Mobile Speed change',
#                    labels={'count':'Frequency','value':'$\Delta Speed (Kpbs)$','variable':'property'},
#                    nbins=100)
# fig.show()

# fig = px.histogram(Broadband_Stats['Change_Download'],title='Frequency distribution of Broadband Speed change',
#                    labels={'count':'Frequency','value':'$\Delta Speed (Kpbs)$','variable':'property'},
#                    nbins=100)
# fig.show()
Total_Stats = pd.concat([Broadband_Stats['Change_Download'],Mobile_Stats['Change_Download']],axis=1)
Total_Stats.columns=['Mobile','Broadband']

# STEFANOS: Disable plotting
# fig = go.Figure(data=[go.Histogram(x=Broadband_Stats['Change_Download'],opacity=0.65,name='Broadband')])
# fig.add_trace(go.Histogram(x=Mobile_Stats['Change_Download'],opacity=0.65,name='Mobile'))
# fig.update_layout(barmode='overlay',
#                   title='Frequency Distribution of Speed change',
#                   xaxis_title="$\Delta\ Speed\ (Kbps)$", yaxis_title="Number of Countries",
#                   legend_title='Color')
# fig.show()

time: 2.38 s (started: 2024-05-07 14:49:05 -04:00)


It can see that most of the countries changed between -5000 Kbps to 5000 kbps. A common graph for all the countries is possible but makes it difficult to understand. Therefore it is better we split the countries into different visualizations, for seperate degrees of change

In [102]:
# STEFANOS: Disable plotting
# px.bar(Mobile_Stats,y='Change_Download',labels={'Name':'Country','Change_Download':'Observed Change'},title='Summary of all changes 2020 Q1 - 2022 Q2 ')

time: 201 µs (started: 2024-05-07 14:49:08 -04:00)


In [103]:
#ImprovedCountries_M = Mobile_Stats[(Mobile_Stats['Change_Download'] < 3000) &
#                                (Mobile_Stats['Change_Download'] >0)]
#px.bar(ImprovedCountries_M,y='Change_Download',labels={'Name':'Country','Change_Download':'Improved Download Speed'},title='Countries that improved download speeds')

ImprovedCountries_B = Broadband_Stats[(Broadband_Stats['Change_Download'] < 3000) &
                                (Broadband_Stats['Change_Download'] > 0)]

# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Speed change',
#                   xaxis_title="Country", yaxis_title="Improved Speed (Kbps)",
#                   legend_title='Color')
# fig.show()

time: 7.63 ms (started: 2024-05-07 14:49:08 -04:00)


In [104]:
#ImprovedCountries2 = Mobile_Stats[(Mobile_Stats['Change_Download'] >= 10000)]
#px.bar(ImprovedCountries2,y='Change_Download',labels={'Name':'Country','Change_Download':'Improved Download Speed'},title='Countries that improved download speeds')
#ImprovedCountries_M = Mobile_Stats[(Mobile_Stats['Change_Download'] < 8000) &
#                                (Mobile_Stats['Change_Download'] >3000)]
#px.bar(ImprovedCountries_M,y='Change_Download',labels={'Name':'Country','Change_Download':'Improved Download Speed'},title='Countries that improved download speeds')

ImprovedCountries_B = Broadband_Stats[(Broadband_Stats['Change_Download'] < 8000) &
                                (Broadband_Stats['Change_Download'] > 3000)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Speed change',
#                   xaxis_title="Country", yaxis_title="Improved Speed (Kbps)",
#                   legend_title='Color')
# fig.show()

time: 12.5 ms (started: 2024-05-07 14:49:08 -04:00)


In [105]:
ImprovedCountries_B = Broadband_Stats[(Broadband_Stats['Change_Download'] < 16000) &
                                (Broadband_Stats['Change_Download'] > 8000)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Speed change',
#                   xaxis_title="Country", yaxis_title="Improved Speed (Kbps)",
#                   legend_title='Color')
# fig.show()

time: 13.1 ms (started: 2024-05-07 14:49:08 -04:00)


In [106]:
ImprovedCountries_B = Broadband_Stats[(Broadband_Stats['Change_Download'] < 60000) &
                                (Broadband_Stats['Change_Download'] > 16000)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Speed change',
#                   xaxis_title="Country", yaxis_title="Improved Speed (Kbps)",
#                   legend_title='Color')
# fig.show()

time: 11.3 ms (started: 2024-05-07 14:49:08 -04:00)


In [107]:
ImprovedCountries_B = Broadband_Stats[(Broadband_Stats['Change_Download'] >= 60000)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Speed change',
#                   xaxis_title="Country", yaxis_title="Improved Speed (Kbps)",
#                   legend_title='Color')
# fig.show()

time: 11.7 ms (started: 2024-05-07 14:49:08 -04:00)


In [108]:
#DeterioratedSpeeds = Mobile_Stats[(Mobile_Stats['Change_Download'] < 0 )]
#px.bar(DeterioratedSpeeds,y='Change_Download',labels={'Name':'Country','Change_Download':'Improved Download Speed'},title='Decreasing Countries\' download speeds')
ImprovedCountries_B = Broadband_Stats[(Broadband_Stats['Change_Download'] < 0)]
Countries = ImprovedCountries_B.index

# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Speed change',
#                   xaxis_title="Country", yaxis_title="Improved Speed (Kbps)",
#                   legend_title='Color')
# fig.show()

time: 5.2 ms (started: 2024-05-07 14:49:08 -04:00)


In [109]:
Mobile_Stats.sort_values(by=['Change_Download'])

,Change_Download,Change_Upload,Change_Latency
Name,,,
Svalbard,-60722,-7622,26
Cook Islands,-53299,-12437,96
Kiribati,-16481,-6388,323
Cuba,-13664,-3164,22
Lesotho,-10968,149,2
...,...,...,...
Bahrain,112317,7053,1
United Arab Emirates,113202,6452,3
Kuwait,120812,9258,-2


time: 19.2 ms (started: 2024-05-07 14:49:08 -04:00)


In [110]:
Broadband_Stats.sort_values(by=['Change_Download'])

,Change_Download,Change_Upload,Change_Latency
Name,,,
Marshall Islands,-9495,-2082,-13
Comoros,-7547,-18156,38
United States Minor Outlying Islands,-7465,-18425,461
Guinea-Bissau,-5363,-2684,30
Cape Verde,-4525,-8908,-10
...,...,...,...
France,106223,82203,-10
Uruguay,109183,21217,-18
China,127467,23571,3


time: 5.78 ms (started: 2024-05-07 14:49:08 -04:00)


Different graphs are used to show different degrees of Average download speed change for each country. These are the capacities by which countries have changed. 

From the above graphs it can be seen that **China** has improved the most average (boardband) download speed and **Antarctica** has lost the most average download internet speed (broadband). Meanwhile **Korea** has the highest improvement in Mobile internet download speed, while **Cook Islands** has lost the most average mobile download speed

These metrics can be misleading as they show average speed change for the whole country. The infrastructure investment/deterioration of the country can only be known after the number is multiplied by population of the country.

## Percentage Download speed Visualization

In this view, we have normalized all improvements or depreciation to the original value, therefore large countries such as china which already have large infrastructure will be given less weightage and small countries that are developing would be given more preference. This view can be used everywhere as the values are normalized to percentages. Therefore larger internet speed nations will be given lower priority to the smaller speed nations. This is NOT a metric of a nation's total bandwidth as that will require denormalization with the nation's population. This metric can be used to compare the rate of national speed improvement between nations.

In [111]:
Mobile_Stats_relative = Mobile_df.groupby('Name').agg(
    Change_Download=('Avg. Avg D Kbps', lambda x: (list(x)[-1] - list(x)[0])/list(x)[0]),
    Change_Upload=('Avg. Avg U Kbps', lambda x: (list(x)[-1] - list(x)[0])/list(x)[0]),
    Change_Latency=('Avg Lat Ms', lambda x: (list(x)[-1] - list(x)[0])/list(x)[0])
)
Broadband_Stats_relative = Broadband_df.groupby('Name').agg(
    Change_Download=('Avg. Avg D Kbps', lambda x: (list(x)[-1] - list(x)[0])/list(x)[0]),
    Change_Upload=('Avg. Avg U Kbps', lambda x: (list(x)[-1] - list(x)[0])/list(x)[0]),
    Change_Latency=('Avg Lat Ms', lambda x: (list(x)[-1] - list(x)[0])/list(x)[0])
)

time: 3.19 s (started: 2024-05-07 14:49:08 -04:00)


In [112]:
ImprovedCountries_B = Broadband_Stats_relative[(Broadband_Stats_relative['Change_Download'] >= 2)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats_relative.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Relative Speed change',
#                   xaxis_title="Country", yaxis_title="Relative Improvement of Speed",
#                   legend_title='Color')
# fig.show()

time: 1.27 ms (started: 2024-05-07 14:49:11 -04:00)


In [113]:
ImprovedCountries_B = Broadband_Stats_relative[(Broadband_Stats_relative['Change_Download'] >= 1) & (Broadband_Stats_relative['Change_Download'] < 2)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats_relative.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Relative Speed change',
#                   xaxis_title="Country", yaxis_title="Relative Improvement of Speed",
#                   legend_title='Color')
# fig.show()

time: 11.7 ms (started: 2024-05-07 14:49:11 -04:00)


In [114]:
ImprovedCountries_B = Broadband_Stats_relative[(Broadband_Stats_relative['Change_Download'] >= 0.5) & (Broadband_Stats_relative['Change_Download'] < 1)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats_relative.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Relative Speed change',
#                   xaxis_title="Country", yaxis_title="Relative Improvement of Speed",
#                   legend_title='Color')
# fig.show()

time: 10.6 ms (started: 2024-05-07 14:49:11 -04:00)


In [115]:
ImprovedCountries_B = Broadband_Stats_relative[(Broadband_Stats_relative['Change_Download'] >= 0.2) & (Broadband_Stats_relative['Change_Download'] < 0.5)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats_relative.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Relative Speed change',
#                   xaxis_title="Country", yaxis_title="Relative Improvement of Speed",
#                   legend_title='Color')
# fig.show()

time: 12.2 ms (started: 2024-05-07 14:49:11 -04:00)


In [116]:
ImprovedCountries_B = Broadband_Stats_relative[(Broadband_Stats_relative['Change_Download'] >= 0) & (Broadband_Stats_relative['Change_Download'] < 0.2)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats_relative.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Relative Speed change',
#                   xaxis_title="Country", yaxis_title="Relative Improvement of Speed",
#                   legend_title='Color')
# fig.show()

time: 11 ms (started: 2024-05-07 14:49:11 -04:00)


In [117]:
ImprovedCountries_B = Broadband_Stats_relative[(Broadband_Stats_relative['Change_Download'] < 0)]
# STEFANOS: Disable plotting
# fig = go.Figure()
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=ImprovedCountries_B['Change_Download'],opacity=0.6,name='Broadband'))
# fig.add_trace(go.Bar(x=ImprovedCountries_B.index,y=Mobile_Stats_relative.query("index in @ImprovedCountries_B.index")['Change_Download'],opacity=0.6,name='Mobile'))
# fig.update_layout(barmode='group',
#                   title='Bar Chart of Relative Speed change',
#                   xaxis_title="Country", yaxis_title="Relative Improvement of Speed",
#                   legend_title='Color')
# fig.show()

time: 12.1 ms (started: 2024-05-07 14:49:11 -04:00)


In [118]:
Broadband_Stats_relative.sort_values(by=['Change_Download'])

,Change_Download,Change_Upload,Change_Latency
Name,,,
Marshall Islands,-0.838855,-0.435747,-0.141304
Comoros,-0.726721,-0.973147,0.236025
Guinea-Bissau,-0.492244,-0.403185,0.405405
United States Minor Outlying Islands,-0.406104,-0.920698,1.945148
Saint Helena,-0.399552,-0.162749,-0.015524
...,...,...,...
Nauru,6.308000,23.425648,-0.641304
Tonga,6.369925,5.602930,-0.219388
Wallis and Futuna Islands,7.068718,8.957143,-0.109718


time: 15 ms (started: 2024-05-07 14:49:11 -04:00)


In [119]:
Mobile_Stats_relative.sort_values(by=['Change_Download'])

,Change_Download,Change_Upload,Change_Latency
Name,,,
Wallis and Futuna Islands,-0.991780,-0.973705,0.237410
Cook Islands,-0.821603,-0.653204,3.692308
Kiribati,-0.801917,-0.697532,1.362869
Cuba,-0.567984,-0.303181,0.176000
Timor-Leste,-0.543723,-0.483994,-0.027027
...,...,...,...
Cayman Islands,3.223961,1.128062,-0.026316
Mauritania,3.328165,4.981119,0.153846
Guinea-Bissau,3.554574,1.512042,-0.583333


time: 6.15 ms (started: 2024-05-07 14:49:11 -04:00)


In [120]:
SNAPPY_end_time = time.perf_counter_ns()
print("Total elapsed time:", (SNAPPY_end_time - SNAPPY_start_time) / (10 ** 9))

Total elapsed time: 8.165380904
time: 378 µs (started: 2024-05-07 14:49:11 -04:00)


In this view we have **Saint Pierre and Miquelon** improving the most relative to its baseline speed and **Wallis and Futuna Islands** losing almost all of its original national average download speed for mobile devices. On the other hand, we have **Antarctica** losing 0.82x its original average speed while Tongo improve its speed by 5x for broadband downloads. Either ends of this view of data would be filled with countries that have low download speeds to begin with, as in that case a minute improvement would also be amplified due to its relative nature.